In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

from dataset_helper import get_mnist_data

import sys
sys.path.append('../mlrose')
from mlrose_hiive import NeuralNetwork

In [ ]:
LR = 1
ALG = 'random_hill_climb'
MAX_ATTEMPTS = 100
CLIP = 5
HIDDEN_NODES = [10,]

In [ ]:
X, y, y_one_hot = get_mnist_data()

In [ ]:
# Split data into training and test sets
skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=True)
train_index, test_index = list(skf.split(X, y))[0]
X_train = X[train_index]
X_test = X[test_index]
y_train = y[train_index]
y_test = y[test_index]
y_train_one_hot = y_one_hot[train_index]
y_test_one_hot = y_one_hot[test_index]

In [ ]:
X_train.shape

In [ ]:
skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=True)
data = {
    'lr': LR,
    'alg': ALG,
    'max_attempts': MAX_ATTEMPTS,
    'clip': CLIP,
    'hidden_nodes': HIDDEN_NODES,
    'runs': [],
}
for run_i, (train_index, test_index) in enumerate(skf.split(X, y)):
    run_data = {}

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    y_train_one_hot = y_one_hot[train_index]
    y_test_one_hot = y_one_hot[test_index]

    iter_list = np.arange(1, 10001, 100) 
    test_scores = []
    for iterations in tqdm(iter_list):
        nn_model = NeuralNetwork(
            hidden_nodes=HIDDEN_NODES, activation='relu', 
            algorithm=ALG, 
            max_iters=iterations, bias=True, is_classifier=True, 
            learning_rate = 1, early_stopping=True, clip_max=CLIP, max_attempts=MAX_ATTEMPTS, 
            random_state=run_i, curve=True
        )
        nn_model.fit(X_train, y_train_one_hot)
        score = nn_model.score(X_test, y_test_one_hot)
        test_scores.append(score)
        if len(nn_model.fitness_curve) < iterations:
            # print("Early stopping at ", iterations)
            break
    run_data['test_scores'] = test_scores
    run_data['iterations'] = iter_list[:len(test_scores)]
    run_data['fitness_curve'] = nn_model.fitness_curve

    data['runs'].append(run_data)

In [ ]:
path = f'metrics/{ALG}/{ALG}_lr{LR}_clip{CLIP}_max_attempts{MAX_ATTEMPTS}_hidden_nodes{HIDDEN_NODES}.pkl'
os.makedirs(os.path.dirname(path), exist_ok=True)
pickle.dump(data, open(path, 'wb'))